In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
import json
import os

from openai import OpenAI

In [17]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY is not None, "OPENAI_API_KEY environment variable is not set"

In [18]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [19]:
def get_llm_response_openai(model, messages, output_format="text"):
    response = client.responses.create(model=model, input=messages)
    if output_format == "json":
        try:
            return json.loads(response.output_text)
        except json.JSONDecodeError as e:
            print("Error parsing JSON response:", e)
            return None
    return response.output_text

In [20]:
def write_poem(model: str, topic: str, word_count: str | int):
    messages = [
        {
            "role": "developer",
            "content": "You are a helpful assistant that writes poems with user-specified word counts. Structure the output as a JSON object with keys 'title' and 'body'. Ensure the poem has exactly the specified number of words.",
        },
        {
            "role": "user",
            "content": f"Write a poem in {word_count} words about {topic}.",
        },
    ]
    return get_llm_response_openai(model, messages, output_format="json")

In [21]:
def get_word_count(poem):
    return len(poem.strip().split())

In [22]:
def reflect_on_poem(model, poem, topic, word_count):
    poem_body = poem.get("body", "") if isinstance(poem, dict) else ""
    word_count = get_word_count(poem_body)
    messages = [
        {
            "role": "developer",
            "content": "You are a thoughtful assistant that reflects on poems. Provide insights about the poem's themes, and style. Structure your response as a JSON object with keys 'feedback_on_topic', 'feedback_on_word_count', and 'improvement_ideas'. Specify feedback on topic adherence in 'feedback_on_topic' and word count accuracy in 'feedback_on_word_count'.",
        },
        {
            "role": "user",
            "content": f"The following poem has {word_count} words.\n\n{poem}\n\n User asked for a poem about {topic}. Reflect on how well the poem captures the essence of {topic}. User asked for {word_count} words, does the poem meet this requirement?",
        },
    ]
    return get_llm_response_openai(model, messages, output_format="json")

In [23]:
def improve_poem(model, poem, feedback, topic, word_count):
    messages = [
        {
            "role": "developer",
            "content": "You are a skilled poet and editor. Use the provided feedback to enhance the poem while maintaining its original style and themes. Structure the output as a JSON object with keys 'title' and 'body'.",
        },
        {
            "role": "user",
            "content": f"User requested for a poem about {topic} in {word_count} words.\n\nHere is the original poem:\n\n{poem}\n\nHere is the feedback:\n\n{feedback}\n\nPlease revise the poem accordingly.",
        },
    ]
    return get_llm_response_openai(model, messages, output_format="json")

In [24]:
def my_poet_agent(
    poet_model: str,
    critic_model: str,
    topic: str,
    word_count: str | int,
):
    first_draft = write_poem(
        model=poet_model,
        topic=topic,
        word_count=word_count,
    )
    poem_feedback = reflect_on_poem(
        model=critic_model,
        poem=first_draft,
        topic=topic,
        word_count=word_count,
    )
    final_poem = improve_poem(
        model=poet_model,
        poem=first_draft,
        feedback=poem_feedback,
        topic=topic,
        word_count=word_count,
    )
    return first_draft, poem_feedback, final_poem

In [25]:
topic = "sunday mornings"
word_count = "sqrt(444)"
poet_model = "gpt-5-nano-2025-08-07"
critic_model = "gpt-5-2025-08-07"

In [26]:
first_draft, poem_feedback, final_poem = my_poet_agent(
    poet_model=poet_model,
    critic_model=critic_model,
    topic=topic,
    word_count=word_count,
)

In [27]:
print(first_draft.get("title", ""), end="\n\n")
print(first_draft.get("body", ""))

Sunday Morning Hush

Sunday morning hush settles like warm light, slippers sigh, kettle hums, windows glow, time softens, promises drift open into quiet hours.


In [28]:
print(poem_feedback.get("feedback_on_topic", ""), end="\n\n")
print(poem_feedback.get("feedback_on_word_count", ""), end="\n\n")
print(poem_feedback.get("improvement_ideas", ""))

Strongly aligned with Sunday mornings. The poem evokes a calm, domestic ritual through sensory details: the sibilant hush, slippers, kettle, and glowing windows. Personification (“slippers sigh,” “time softens,” “promises drift open”) deepens the mood of ease and renewal. The tone is gentle and unhurried, capturing the hushed, liminal feel of a Sunday start.

The body contains 21 words, meeting the requirement. Note: if the title were included in the count, the total would be 24 words.

['Introduce a distinctly Sunday marker (e.g., church bells, no alarm, the Sunday paper, pancakes) to sharpen the setting.', 'Add a scent image (fresh coffee, toast, rain on pavement) to round out the sensory field.', 'Consider a small external detail (distant bells, a quiet street) to widen the scene beyond the room.', 'Play with line breaks or a subtle pause (an em dash or period) to mimic the spacious Sunday tempo.', 'Offer a gentle turn at the end (a held breath before the day, a shared glance) to le

In [29]:
print(final_poem.get("title", ""), end="\n\n")
print(final_poem.get("body", ""))

Sunday Morning Hush

Sunday morning hush settles; slippers sigh, kettle hums, coffee steams, motes glitter; distant bells toll; paper rustles, pancakes warm; shared smile.


In [30]:
final_word_count = get_word_count(final_poem.get("body", ""))
final_word_count

21

In [ ]:
first_draft = write_poem(
    model=poet_model,
    topic=topic,
    word_count=word_count,
)
print(first_draft.get("title", ""), end="\n\n")
print(first_draft.get("body", ""))

In [ ]:
poem_feedback = reflect_on_poem(
    model=critic_model,
    poem=first_draft,
    topic=topic,
    word_count=word_count,
)
print(poem_feedback.get("feedback_on_topic", ""), end="\n\n")
print(poem_feedback.get("feedback_on_word_count", ""), end="\n\n")
print(poem_feedback.get("improvement_ideas", ""))

In [ ]:
final_poem = improve_poem(
    model=poet_model,
    poem=first_draft,
    feedback=poem_feedback,
    topic=topic,
    word_count=word_count,
)
print(final_poem.get("title", ""), end="\n\n")
print(final_poem.get("body", ""))

In [ ]:
final_word_count = get_word_count(final_poem.get("body", ""))
final_word_count